1. 일반 로지스틱
0) 이 노트북의 목표 구조
목표: dependent_ecotype (0/1) 이진 분류를 로지스틱으로 학습하고,
GridSearchCV로 하이퍼파라미터 튜닝
테스트셋에서 성능 평가 (AUC 중심)
learning curve로 과적합 여부 판정

(옵션) 변수 중요도 / SHAP 해석

1) 라이브러리 로딩
핵심 패키지
pandas / numpy → 데이터 핸들링
sklearn
train_test_split, GridSearchCV, learning_curve
Pipeline, ColumnTransformer
SimpleImputer, OneHotEncoder, StandardScaler
LogisticRegression
metrics (accuracy, precision, recall, f1, roc_auc, confusion_matrix 등)

**기억 포인트
분류에서 성능 비교 기준을 roc_auc로 통일해 둔 것이 좋음 (불균형 대응에 강함)**

2) 함수 정의 (노트북 상단 유틸 함수들)
노트북에는 다음 기능들이 정의되어 있음:
데이터 품질 확인 (수치형 describe / 결측 / 분포 등)
범주형 변수 분포 확인
permutation importance
학습곡선 기반 과적합 판정 함수
성능평가 + 과적합 판정 통합 함수
SHAP 분석 함수
기억 포인트 (중요)

학습곡선 함수가 원래 회귀용(neg RMSE) 기반이라면,
분류에서 roc_auc 사용할 때는 부호(-) 처리하면 안 됨
AUC가 음수로 나오면 계산이 잘못된 것임

3) 데이터 로드 + 타입 처리
(1) 데이터 불러오기
3.coding_book_mapping.csv 로드

(2) 명목형 변수 category 정리
object / category 타입 정리

**꼭 기억할 것**

종속변수 dependent_ecotype까지 category로 두면 지표 계산 시 오류 가능
따라서 반드시 아래처럼 고정

y = df3[yname].astype(int)
(3) 다른 종속변수 제거

예: dependent_wage_work 등 다른 타깃 변수는 제거하여 데이터 누수 방지

4) 데이터 품질 확인 (EDA)

수치형 변수 결측률 확인

범주형 변수 레벨 및 결측 확인

기억 포인트

결측률 높은 변수는
→ 이후 “결측 50% 이상 제거 버전” 실험 시 핵심 비교 대상

5) 전처리 + 데이터 분할 (가장 중요)
train/test split
x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size=0.25,
    random_state=52,
    stratify=y
)
꼭 기억

분류에서는 stratify=y 거의 필수
클래스 비율이 깨지면 AUC/F1 왜곡 가능

6) 전처리를 파이프라인 안으로 넣는 정석 구조
(A) 수치형 처리

결측 → median

스케일링 → StandardScaler

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
(B) 범주형 처리

결측 → "Missing"

더미화 → OneHotEncoder

drop="first" → 다중공선성 완화

handle_unknown="ignore" → 테스트셋 새로운 범주 대응

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="Missing")),
    ("onehot", OneHotEncoder(drop="first", handle_unknown="ignore"))
])
(C) ColumnTransformer 결합
preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols),
])
매우 중요한 포인트

get_dummies, reindex 필요 없음

CV 내부에서 fold마다 imputer/onehot 재학습
→ 교차검증 점수 누수 방지

7) 모델링: 로지스틱 + GridSearchCV
pipe = Pipeline([
    ("preprocess", preprocess),
    ("model", LogisticRegression(random_state=52))
])
탐색 파라미터

C

max_iter

class_weight

solver="lbfgs", penalty="l2"

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1
)
기억 포인트

class_weight="balanced" 포함은 매우 적절
→ 불균형 데이터에서 recall 개선 가능

8) 테스트셋 성능 평가
핵심 원칙

파이프라인 사용 시 x_test 원본 그대로 사용
(x_test2 같은 더미 데이터 사용 금지)

평가 지표

Confusion Matrix (tn, fp, fn, tp)

Accuracy

Precision

Recall

F1

AUC (가장 중요)

FPR, TNR

꼭 기억

AUC는 반드시 predict_proba()[:, 1] 사용

log_loss는 음수 붙이지 않음

9) 과적합 확인 (Learning Curve)
핵심 원칙

learning_curve에도 원본 x_train 사용

AUC 기반일 때

scoring="roc_auc"

AUC는 음수 처리 금지

판정 기준 예시

Train AUC - CV AUC > 0.1 → 과대적합 의심

CV 표준편차가 큼 → 데이터 부족

gap 작고 std 작음 → 일반화 양호

10) 해석 (선택)

permutation importance → 전체 중요 변수 확인

SHAP → 방향성 + 상호작용 확인

기억 포인트

로지스틱은 coef_ 기반 해석 가능

트리/부스팅 모델에서는 SHAP 활용도가 더 높음

모델별 전처리 템플릿 요약
로지스틱

수치형: median + scaling

범주형: Missing + one-hot

split: stratify

tuning: GridSearchCV(AUC)